# Websites Phishing Detection using Logistic Regresión and the KNN algorithm

## Task 1 | Regresión Lineal

### 1. Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y w₂. Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación del coeficiente 3w₂ en el contexto del modelo?


### 2. Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la multicolinealidad a la interpretación de los coeficientes de regresión individuales?



## Análisis Exploratorio

In [46]:
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [47]:
df = pd.read_csv('data/dataset_phishing.csv')
df.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [48]:
features = df[[
    'url', 
    'length_url', 
    'length_hostname', 
    'ip', 
    # 'nb_www', 
    # 'nb_com',
    # 'nb_dslash', 
    'http_in_path', 
    'https_token', 
    # 'ratio_digits_url',
    # 'ratio_digits_host', 
    'punycode', 
    'port', 
    'tld_in_path',
    'tld_in_subdomain', 
    'abnormal_subdomain',
    # 'nb_subdomains',
    'prefix_suffix', 
    'random_domain', 
    'shortening_service',
    'path_extension', 
    # 'nb_redirection', 'nb_external_redirection',
    'phish_hints', 
    'domain_in_brand',
    'brand_in_subdomain', 
    'brand_in_path', 
    'suspecious_tld',
    'statistical_report', 
    # 'nb_hyperlinks', 
    # 'ratio_intHyperlinks',
    # 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
    # 'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors',
    # 'ratio_extErrors', 
    'login_form', 
    'external_favicon', 
    # 'links_in_tags',
    # 'submit_email',  0's across the board
    # 'ratio_intMedia', 'ratio_extMedia', 
    # 'sfh', 0's across the board
    'iframe',
    'popup_window', 
    'onmouseover', 
    'right_clic',
    'empty_title', 
    'domain_in_title', 
    'domain_with_copyright',
    'whois_registered_domain', 
    'domain_registration_length', 'domain_age',
    'web_traffic', 'dns_record', 'google_index', 'page_rank', 'status']].copy()

### Encoding

Solamente es necesario en la parte del objetivo, ya que es la única variable categórica.

In [49]:
features['status'] = features['status'].map({'phishing': 1, 'legitimate': 0})

In [50]:
features.describe()

,length_url,length_hostname,ip,http_in_path,https_token,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
count,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,...,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,1.143000e+04,11430.000000,11430.000000,11430.000000,11430.000000
mean,61.126684,21.090289,0.150569,0.016710,0.610936,0.000350,0.002362,0.065617,0.050131,0.021610,...,0.775853,0.439545,0.072878,492.532196,4062.543745,8.567566e+05,0.020122,0.533946,3.185739,0.500000
std,55.297318,10.777171,0.357644,0.169358,0.487559,0.018705,0.048547,0.247622,0.218225,0.145412,...,0.417038,0.496353,0.259948,814.769415,3107.784600,1.995606e+06,0.140425,0.498868,2.536955,0.500022
min,12.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-1.000000,-12.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,33.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,84.000000,972.250000,0.000000e+00,0.000000,0.000000,1.000000,0.000000
50%,47.000000,19.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,242.000000,3993.000000,1.651000e+03,0.000000,1.000000,3.000000,0.500000
75%,71.000000,24.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,449.000000,7026.750000,3.738455e+05,0.000000,1.000000,5.000000,1.000000
max,1641.000000,214.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,29829.000000,12874.000000,1.076799e+07,1.000000,1.000000,10.000000,1.000000


In [51]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   url                         11430 non-null  object
 1   length_url                  11430 non-null  int64 
 2   length_hostname             11430 non-null  int64 
 3   ip                          11430 non-null  int64 
 4   http_in_path                11430 non-null  int64 
 5   https_token                 11430 non-null  int64 
 6   punycode                    11430 non-null  int64 
 7   port                        11430 non-null  int64 
 8   tld_in_path                 11430 non-null  int64 
 9   tld_in_subdomain            11430 non-null  int64 
 10  abnormal_subdomain          11430 non-null  int64 
 11  prefix_suffix               11430 non-null  int64 
 12  random_domain               11430 non-null  int64 
 13  shortening_service          11430 non-null  in

### Balanceo de clases

No necesita de ningún balanceo de clases.

In [52]:
features['status'].value_counts()

status
0    5715
1    5715
Name: count, dtype: int64

### Scaling

Las magnitudes de los datos son muy diferentes, por lo que es necesario escalarlos.
Serán los siguientes features: length_url, length_hostname, domain_registration_length, domain_age, web_traffic, page_rank

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [54]:
# List of features to scale
features_to_scale = ['length_url', 'length_hostname', 'domain_registration_length', 'domain_age', 'web_traffic', 'page_rank']

# Apply the scaler to the features in the list
features[features_to_scale] = scaler.fit_transform(features[features_to_scale])

In [55]:
features[features_to_scale].mean()

length_url                   -5.221836e-17
length_hostname              -1.367624e-16
domain_registration_length    2.237930e-17
domain_age                   -1.243294e-18
web_traffic                  -1.429789e-17
page_rank                    -8.703061e-18
dtype: float64

### Data Splitting

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
X = features.drop('status', axis=1)
y = features['status']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21562)

## Task 2.1 | Regresión Logística

### Entrenamiento sin librerías

### Entrenamiento con librerías

## Task 2.2 | KNN

### Entrenamiento sin librerías

### Entrenamiento con librerías